In [39]:
import tensorflow as tf
import numpy as np
from typing import List
import numpy as np
from ExperimentConfig import ExperimentConfig
import tensorflow as tf


class ResultMemory:
    def __init__(self, max_size: int):
        self.data = np.zeros(shape=(max_size, 5))
        self.max_size = max_size
        self.count = 0
    def add_sample(self, sample: np.ndarray):
        sr = self.count % self.max_size
        self.count += sample.shape[0]

        er = sr + sample.shape[0]
        self.data[sr:er] = sample

    def get_batch(self, batch_size) -> (np.ndarray, np.ndarray, np.ndarray):
        indices = np.random.randint(0, min(self.max_size, self.count), batch_size)
        return tf.gather(self.data, indices)


def create_batch_from_results(config: ExperimentConfig, memory: ResultMemory) -> (np.ndarray, np.ndarray, np.ndarray):

    batch_size = config.batch_size
    results = memory.get_batch(batch_size)

    if results.ndim == 1:
        results = np.expand_dims(results, 0)

    in_rows = results.shape[0]
    cols = results.shape[1]
    out_rows = in_rows * 2

    bin_it = lambda x: config.convert_ideology_to_bin(x)
    v_bin = np.vectorize(bin_it)
    results = v_bin(results)

    x = np.zeros(shape=(out_rows , config.n_bins), dtype=np.single)
    y = np.zeros(shape=(out_rows , config.n_bins), dtype=np.single)
    mask = np.zeros(shape=(out_rows , config.n_bins), dtype=np.single)


    out_row = 0
    for in_row in range(in_rows):
        i = np.random.randint(0, cols)
        for j in range(cols):
            if i != j:
                x[out_row, results[in_row, j]] = 1

        if i == 0:
            y[out_row, results[in_row, i]] = 1
        mask[out_row, results[in_row, i]] = 1
        out_row += 1

    x[out_row: out_rows] = np.flip(x[0: out_row], axis = 1)
    y[out_row: out_rows] = np.flip(y[0: out_row], axis = 1)
    mask[out_row: out_rows] = np.flip(mask[0: out_row], axis = 1)
    return x, y, mask

In [40]:
t_config = ExperimentConfig("test", "IRV")
result_memory = ResultMemory(100000)
for i in range(10000):
    cc = t_config.gen_candidates(5)
    t = t_config.create_training_sample(cc, cc[2])
    result_memory.add_sample(t)

x, y, mask = create_batch_from_results(t_config, result_memory)

print(x)
print(y)
print(mask)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [41]:
x.shape


(4096, 21)